In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras as keras

import cv2
import numpy as np

In [2]:
video_ids = [2] # range(1, 6)
frame_skip = 8

In [3]:
labels = []
for id in video_ids:
    with open(f'data/labels/video{id}.txt') as f:
        lines = f.readlines()
        lines = [line.split() for line in lines]
        lines = np.array(lines)

        # Remove OTH label
        lines = lines[lines[:, 1] != 'OTH']
        
        # Prune frames
        lines = lines[::frame_skip]

        labels.append(lines)

labels = np.array(labels)

In [4]:
data = []
for id, labels in zip(video_ids, labels):
    video = cv2.VideoCapture(f"data/videos/video{id}.mp4")
    for i, frame_num in enumerate(labels[:, 0].astype(int)):
        print(f'Loading video {id}... ({i}/{labels.shape[0]})', end='\r')
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = video.read()
    
        data.append(frame)

    video.release()

data = np.array(data)

In [5]:
# One-hot encode labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(labels[:, 1])
labels = label_encoder.transform(labels[:, 1])

labels = keras.utils.to_categorical(labels)

In [6]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose, BatchNormalization, Dropout

model = tf.keras.models.Sequential([
    Conv2D(128, (3, 3), activation='relu', input_shape=(720, 1280, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(momentum=0.9),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(512, activation='relu'),
    Dropout(0.2),

    Dense(10, activation='softmax'),
])

model.summary()


2023-12-14 00:41:58.480068: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-12-14 00:41:58.480095: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2023-12-14 00:41:58.480099: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2023-12-14 00:41:58.480138: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-14 00:41:58.480152: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 718, 1278, 128)    3584      
                                                                 
 max_pooling2d (MaxPooling2  (None, 359, 639, 128)     0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 357, 637, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 178, 318, 128)     0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 176, 316, 64)      73792     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 88, 158, 64)       0

In [7]:
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


2023-12-14 00:42:00.560770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


: 